In [31]:
import urllib.request
import gzip
import json

DATA_URL = "http://ciir.cs.umass.edu/downloads/poetry/poetry50k.dedup.jsonl.gz"
data_50k = []


def read_data(target):
    handle = urllib.request.urlopen(DATA_URL)
    with gzip.GzipFile(fileobj=handle) as fp:
        for line in fp:
#             if len(data_50k) > 100:
#                 break
            content = json.loads(line)
            if content["score"]>0.65:
                target.append(content["text"]) 
                
read_data(data_50k)

In [4]:
# data_50k = data_50k[:50000]

In [5]:
# data_50k[0]

'^Dedication\nHere\ts\ta\trhyme\tfor\tBarbara,\nLaughing\twhite\tand\tpink,\nHere\ts\ta\trhyme\tfor\tsmiling\tTed,\nAnd\tone\tfor\tWink.\nNow\tDick\ts\tnot\tmuch\tat\treading\trhymes,\nHe\td\trather\tsit\tand\tfish.\nWell,\there\ts\ta\tcouple\tof\tverses,\tDick,\nRead\tthem\tif\tyou\twish!\n479468\n'

In [28]:
data_1000=data_50k[:100]

In [ ]:
import re
line_break = re.compile(r"(\n|<br />)")

from collections import defaultdict, Counter


def build_dictionary(database):
    
    dictionary = {}
    sentence_length_dictionary={}
    
    punctuations = '''!()-+='[]{};:"\,<>./?@#$%^&*_~1234567890«»|'''
    translator = str.maketrans('', '', punctuations)
    prev_words=["","","",""]

    for poem in database:
        lines = re.split(line_break, poem)

        # first lines are usually titles
        for line in lines[1:]:    
            line = line.replace("\n","")
            line = line.translate(translator)
            newline = True          
            index = 0
            words = line.split('\t')
            for word in words:
                #print(prev_words[::-1])
                if word:
                    if not word == "I":
                        word = word.lower()

                    if newline:
                        if len(words) in sentence_length_dictionary:
                            sentence_length_dictionary[len(words)] +=1
                        else:
                            sentence_length_dictionary[len(words)] = 1
                        for _ in prev_words:
                            _ = ""
                        newline = False

                    if index>=1 and not prev_words[0] == "":
                        key = str((prev_words[0]))
                        if key in dictionary:
                            dictionary[key]["frequency"] += 1
                            dictionary[key]["dict"][word] += 1
                        else:
                            counter = Counter()
                            counter[word] +=1 
                            dictionary[word] = {
                                                "frequency":1,
                                                "dict": counter
                                                }

                    if index >= 2 and not prev_words[1] == "":
                        key = str((prev_words[1],prev_words[0]))
                        if key in dictionary:
                            dictionary[key]["frequency"] += 1
                            dictionary[key]["dict"][word] += 1
                        else:
                            counter = Counter()
                            counter[word] +=1 
                            dictionary[key] = {
                                                "frequency":1,
                                                "dict": counter
                                                }

                    if index >= 3 and not prev_words[2] == "":
                        key = str((prev_words[2], prev_words[1], prev_words[0]))
                        if key in dictionary:
                            dictionary[key]["frequency"] += 1
                            dictionary[key]["dict"][word] += 1
                        else:
                            counter = Counter()
                            counter[word] +=1 
                            dictionary[key] = {
                                                "frequency":1,
                                                "dict": counter
                                                }
                                
                             
                    if index >= 4 and not prev_words[3] == "":
                        key = str((prev_words[3], prev_words[2], prev_words[1], prev_words[0]))
                        if key in dictionary:
                            dictionary[key]["frequency"] += 1
                            dictionary[key]["dict"][word] += 1
                        else:
                            counter = Counter()
                            counter[word] +=1 
                            dictionary[key] = {
                                                "frequency":1,
                                                "dict": counter
                                                }

                    
                    for i in range(len(prev_words)-1)[::-1]:
                        prev_words[i+1] = prev_words[i]
                    prev_words[0] = word
                    #print("slide: ", prev_words[::-1])
                    
                    index+=1
                    
    return dictionary, sentence_length_dictionary

dictionary_50k,sl_50k = build_dictionary(data_50k)


In [ ]:
with open('dictionary50k.json',"w") as out_file:  
    json.dump(dictionary_50k, out_file, sort_keys=True, indent=4)

with open('sentence_lengthes_50k.json',"w") as out_file:  
    json.dump(sl_50k, out_file, sort_keys=True, indent=4)
